In [2]:
import sys
from pathlib import Path

In [3]:
!{sys.executable} -m pip install flair

  Using cached lxml-5.1.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached regex-2023.12.25-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.10.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 6.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━

In [135]:
from sklearn.metrics import accuracy_score

In [26]:
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.models import TARSClassifier
from flair.trainers import ModelTrainer

In [29]:
tars = TARSClassifier.load("tars-base")

2024-02-17 00:47:23,715 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model


In [32]:
from flair.data import Corpus
from flair.datasets import FlairDatapointDataset
# training dataset consisting of four sentences (2 labeled as "olahraga" (sports) and 2 labeled as "politik" (politics)
train = FlairDatapointDataset(
    [
        Sentence('BSN Sports LLC')
        .add_label('payee_or_misc', 'payee'),
        Sentence('Samuel L. Catherin')
        .add_label('payee_or_misc', 'payee'),
        Sentence('Five Seasons')
        .add_label('payee_or_misc', 'payee'),
        Sentence('fifty thousand and 00/11')
        .add_label('payee_or_misc', 'misc'),
        Sentence('Official check')
        .add_label('payee_or_misc', 'misc')
    ])
test = FlairDatapointDataset(
    [
        Sentence('Central hudson electric').add_label('payee_or_misc', 'payee'),
        Sentence('Pay to the order of').add_label('payee_or_misc', 'misc')
    ])
# make a corpus with train and test split
corpus = Corpus(train=train, test=test)

2024-02-17 00:52:29,947 No dev split found. Using 0% (i.e. 0 samples) of the train split as dev data


In [36]:
corpus.make_label_dictionary(label_type="payee_or_misc")

2024-02-17 00:54:53,433 Computing label dictionary. Progress:


0it [00:00, ?it/s]
5it [00:00, 12710.01it/s]

2024-02-17 00:54:53,439 Dictionary created for label 'payee_or_misc' with 2 values: payee (seen 3 times), misc (seen 2 times)


In [39]:
from flair.trainers import ModelTrainer
# 1. load base TARS
tars = TARSClassifier.load('tars-base')
# 2. make the model aware of the desired set of labels from the new corpus
tars.add_and_switch_to_new_task("PAYEE_OR_MISC", 
                                label_dictionary=corpus.make_label_dictionary("payee_or_misc"),
                                label_type="payee_or_misc")
# 3. initialize the text classifier trainer with your corpus
trainer = ModelTrainer(tars, corpus)

2024-02-17 00:55:25,508 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model
2024-02-17 00:55:25,567 Computing label dictionary. Progress:


0it [00:00, ?it/s]
5it [00:00, 12045.67it/s]

2024-02-17 00:55:25,573 Dictionary created for label 'payee_or_misc' with 2 values: payee (seen 3 times), misc (seen 2 times)


In [40]:
# 4. train model
trainer.train(base_path='resources/taggers/payee', # path to store the model artifacts
              learning_rate=0.02, 
              mini_batch_size=1, 
              max_epochs=10, 
              )

2024-02-17 00:55:51,709 ----------------------------------------------------------------------------------------------------
2024-02-17 00:55:51,711 Model: "TARSClassifier(
  (tars_model): TextClassifier(
    (embeddings): TransformerDocumentEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Lin

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 89.99it/s]

2024-02-17 00:55:57,553 
Results:
- F-score (micro) 0.5
- F-score (macro) 0.3333
- Accuracy 0.5

By class:
              precision    recall  f1-score   support

       payee     0.5000    1.0000    0.6667         1
        misc     0.0000    0.0000    0.0000         1

    accuracy                         0.5000         2
   macro avg     0.2500    0.5000    0.3333         2
weighted avg     0.2500    0.5000    0.3333         2

2024-02-17 00:55:57,554 ----------------------------------------------------------------------------------------------------


{'test_score': 0.5}

In [41]:
# 5. Load few-shot TARS model
tars = TARSClassifier.load('resources/taggers/payee/final-model.pt')

In [56]:
# 6. Prepare a test sentence
sentence = Sentence("Weber County Treasurer")
# 7. Predict for olahraga and politik
tars.predict(sentence, return_probabilities_for_all_classes=True)
print(sentence)

Sentence[3]: "Weber County Treasurer"


## trying wide

In [159]:
# !{sys.executable} -m pip install -q pytorch-widedeep

In [95]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor, TextPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep, TabResnet, BasicRNN
from pytorch_widedeep.metrics import Accuracy

In [100]:
df_train, df_test = pd.read_csv("temp/merged_train.csv"), pd.read_csv("temp/merged_test.csv")

In [101]:
embed_cols = [
    ('payee_txt', 12)
]
cont_cols = ['anchor_x1', 'anchor_y1', 'anchor_x2', 'anchor_y2', 'payee_x1',
       'payee_y1', 'payee_x2', 'payee_y2']

df_train.drop(["idx"], axis=1, inplace=True)
df_test.drop(["idx"], axis=1, inplace=True)

target = df_train["target"].values

In [102]:
text_preprocessor = TextPreprocessor(text_col="payee_txt")

In [ ]:
text_preprocessor

In [103]:
X_text = text_preprocessor.fit_transform(df_train)

The vocabulary contains 160 tokens


In [104]:
tab_preprocessor = TabPreprocessor(continuous_cols=cont_cols,
                                   verbose=1)
X_tab = tab_preprocessor.fit_transform(df_train)

/home/ajkdrag/workspace/ocrtoolkit/venv/lib/python3.8/site-packages/pytorch_widedeep/preprocessing/tab_preprocessor.py:360: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")


In [105]:
# DeepDense: 2 Dense layers
tab_mlp = TabMlp(
    column_idx=tab_preprocessor.column_idx,
    continuous_cols=cont_cols,
    mlp_hidden_dims=[128, 64],
    mlp_dropout=0.1,
)

In [126]:
# DeepText: a stack of 2 LSTMs
basic_rnn = BasicRNN(
    vocab_size=len(text_preprocessor.vocab.itos),
    n_layers=2,
    embed_dim=8,
    hidden_dim=64,
    rnn_dropout=0.5,
)

In [129]:
model = WideDeep(
    deeptabular=tab_mlp,
    deeptext=basic_rnn,
    head_hidden_dims=[256, 128],
)

In [130]:
trainer = Trainer(model, objective="binary", metrics=[(Accuracy)])
trainer.fit(X_tab=X_tab, 
            X_text=X_text,
            target=target, n_epochs=20, 
            batch_size=32, val_split=0.2, validation_freq=2)

valid: 100%|████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 34.63it/s, loss=0.0579, metrics={'acc': 0.9788}]


### test

In [139]:
X_tab_te = tab_preprocessor.transform(df_test)
X_text_te = text_preprocessor.transform(df_test)
preds = trainer.predict(X_tab=X_tab_te, X_text=X_text_te)

predict: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 42.83it/s]


In [142]:
import pickle

In [143]:
# with open("temp/tab_preproc.pkl", "wb") as dp:
#     pickle.dump(tab_preprocessor, dp)

In [144]:
# with open("temp/text_preproc.pkl", "wb") as dp:
#     pickle.dump(text_preprocessor, dp)

In [145]:
# torch.save(model, "temp/model_saved.pt")

In [140]:
np.mean(preds == df_test["target"].values)

0.9766081871345029

In [141]:
accuracy_score(df_test["target"].values, preds)

0.9766081871345029

## xyz

In [55]:
tabmlp = TabMlp(
    mlp_hidden_dims=[200, 100],
    column_idx=tab_preprocessor.column_idx,
    continuous_cols=cont_cols,
    mlp_batchnorm_last=True,
    cont_norm_layer="batchnorm"
)
model = WideDeep(deeptabular=tabmlp)

In [57]:
trainer = Trainer(model, objective="binary", metrics=[(Accuracy)])
trainer.fit(X_tab=X_tab, target=target, n_epochs=15, batch_size=256, val_split=0.2)

valid: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.79it/s, loss=0.246, metrics={'acc': 0.9053}]


In [121]:
tabresnet = TabResnet(
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input, 
    continuous_cols=cont_cols,
    mlp_batchnorm_last=True,
    cont_norm_layer="batchnorm",
    blocks_dims=[200, 100, 100],
    mlp_hidden_dims=[100, 50],
)
model2 = WideDeep(deeptabular=tabresnet)

In [122]:
trainer = Trainer(model2, objective="binary", metrics=[(Accuracy)])
trainer.fit(X_tab=X_tab, target=target, n_epochs=5, batch_size=256, val_split=0.2)

valid: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.48it/s, loss=0.475, metrics={'acc': 0.8316}]


In [107]:
tab_preprocessor2 = TabPreprocessor(
    embed_cols=['payee_txt'],
    continuous_cols=cont_cols,
    cols_to_scale="all",
    with_attention=True)

X_tab2 = tab_preprocessor2.fit_transform(df_train)

In [109]:
tab_preprocessor2.cat_embed_input

[('payee_txt', 749)]

In [131]:
from pytorch_widedeep.models import TabTransformer

tabtransformer = TabTransformer(
    column_idx=tab_preprocessor2.column_idx,
    cat_embed_input=tab_preprocessor2.cat_embed_input, 
    continuous_cols=cont_cols,
    shared_embed=True,
    n_blocks=3,
)
model3 = WideDeep(deeptabular=tabtransformer)

In [132]:
trainer = Trainer(model3, objective="binary", metrics=[(Accuracy)])
trainer.fit(X_tab=X_tab2, target=target, n_epochs=5, batch_size=256, val_split=0.2)

valid: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.96it/s, loss=0.439, metrics={'acc': 0.8316}]


In [125]:
df_test

,anchor_x1,anchor_y1,anchor_x2,anchor_y2,payee_x1,payee_y1,payee_x2,payee_y2,payee_txt,target
0,0.035898,0.209373,0.076653,0.271838,0.307863,0.181312,0.679726,0.281260,Shiva Prasad Kumar veduger канди .,1
1,0.037499,0.221835,0.076550,0.271920,0.307751,0.181188,0.679659,0.281151,Shiva Prasad Kumar veduger канди .,1
2,0.071929,0.296898,0.095270,0.340562,0.114106,0.284404,0.173461,0.337421,SELF,1
3,0.070338,0.303185,0.101585,0.343738,0.114024,0.284368,0.173377,0.337462,SELF,1
4,0.023404,0.215634,0.062554,0.268689,0.084344,0.209328,0.273478,0.262429,ANKIT SHARMA,1
...,...,...,...,...,...,...,...,...,...,...
337,0.020305,0.150076,0.076542,0.199937,0.029726,0.162497,0.062492,0.200041,PAY,0
338,0.020394,0.149926,0.076631,0.199961,0.036004,0.303098,0.496804,0.418716,TRUPEES One Саки IV Thirty five,0
339,0.024913,0.156321,0.067278,0.212457,0.078057,0.574923,0.371906,0.624956,ORIENTAL BANK OF COMMERCE,0
340,0.024970,0.156172,0.067230,0.212470,0.029594,0.162537,0.062435,0.199968,PAY,0


In [60]:
X_tab_te = tab_preprocessor.transform(df_test)
preds = trainer.predict(X_tab=X_tab_te)

predict: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.65it/s]


In [63]:
np.mean(df_test["target"] == preds)

0.9243697478991597

In [65]:
df_test["target"].values

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1])

In [61]:
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0])